In [ ]:
p = np.array([math.sin(x/2)+x/16 for x in range(30)]) + 3# + (np.random.random(L)-0.5)*2
p = torch.from_numpy(p.astype(np.float32))*10
dp = p[1:] - p[:-1]
hold = compute_hold(p)
plt.plot(p, ".-")
plt.plot(hold)
plt.plot(dp)

fsize = 3
features = []
for i in range(p.shape[0]):
    if i < fsize-1:
        features.append(torch.zeros(fsize))
    else:
        features.append(p[i-fsize+1:i+1] - p[i])

features = torch.vstack(features)
for i in range(fsize-1, 10):
    plt.plot(range(i-fsize+1, i+1), features[i].numpy())
plt.grid("on")
plt.tight_layout()

In [ ]:
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import PyConfig
from data_processing.train_dataset import next_price_prediction
from tqdm import tqdm
from ml.models import autoregress_sequense, E2EModel
from e2e_train import E2ETrain
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from backtest import backtest
from loguru import logger

%reload_ext autoreload
%autoreload 2

logger.remove()
logger.add(sys.stderr, level="DEBUG")

def save_model(model):
    model.eval()
    example = torch.rand((1,) + model.inp_shape)
    model_jit = torch.jit.trace(model, example)
    model_jit.save("model.pt")

    # torch.save(model, "model.pt")

In [ ]:
DEVICE = torch.device("cpu")
cfg = PyConfig("zz.py").test()
# cfg.date_end = "2020.01.01"
cfg.date_start = "2023.01.01"

In [ ]:
e2e_train = E2ETrain(cfg)
e2e_train.load_data(dataset_root="/Users/andrybin/Yandex.Disk.localized/fin_data")

In [ ]:
val_size = 0.2
model = e2e_train.train(num_epochs=100, 
                        resume=False,
                        device=DEVICE,
                        val_size=val_size)

In [ ]:
model.eval()
output_seq, result_seq, fee_seq = autoregress_sequense(model, e2e_train.p, e2e_train.features, output_sequense=True, device=DEVICE)
hold = e2e_train.compute_hold()

fig, ax1 = plt.subplots(2, 1, height_ratios = [3, 1])
ax1 = plt.subplot(2, 1, 1)
ax1.plot(result_seq.cumsum(0))
ax1.plot(fee_seq.cumsum(0))
ax1.plot(hold, linewidth=3)

ax1.plot([e2e_train.p.shape[0]*(1-val_size), e2e_train.p.shape[0]*(1-val_size)], 
         [0, max(result_seq.cumsum(0).max(), hold.max())],
         "--",
         linewidth=3)
ax1.grid("on")
plt.tight_layout()
plt.grid("on")

ax2 = ax1.twinx()
ax2.bar(list(range(output_seq.shape[0])), height=output_seq, width=1, alpha=0.4)

ax1 = plt.subplot(2, 1, 2)
ax1.fill(np.hstack([result_seq.cumsum() - hold, np.zeros(1)]))

In [ ]:
result_seq.cumsum() - hold

In [ ]:
with torch.no_grad():   
    model = torch.load("model.pt")
    output = model(torch.from_numpy(e2e_train.features[300:301]))
output

In [ ]:
save_model(model)

In [ ]:
DEVICE = torch.device("cpu")

cfg = PyConfig("zz.py").test()
date_start = datetime.strptime(cfg.date_start, '%Y-%m-%dT%H:%M:%S').strftime('%m/%d/%Y')
date_end = datetime.strptime(cfg.date_end, '%Y-%m-%dT%H:%M:%S').strftime('%m/%d/%Y')

years = [pd.to_datetime(d).date().strftime('%Y-%m-%dT%H:%M:%S') for d in 
         pd.date_range(start=date_start, end=date_end, freq="Y")]

date_start = cfg.date_start
last_prof = 0
for year in years[19:-1]:
    cfg = PyConfig("zz.py").test()
    cfg.date_end = year
    e2e_train = E2ETrain(cfg)
    e2e_train.load_data()
    model = e2e_train.train(num_epochs=200, resume=False, device=DEVICE)
    save_model(model)
    
    cfg = PyConfig("zz.py").test()
    cfg.date_start = year
    brok_results = backtest(cfg)

    plt.plot(brok_results.daily_hist.days, brok_results.daily_hist.profit + last_prof, linewidth=1, color="b", alpha=0.6)
    last_prof += brok_results.final_profit
    plt.grid("on")
    plt.tight_layout()
    # legend.append(f"{cfg.date_start}-{cfg.date_end}")

In [ ]:
brok_results = backtest(cfg)
